In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [38]:
epochs = 10
batch_size = 100
learning_rate = 0.001

In [39]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [40]:
train_dataset = torchvision.datasets.CIFAR10(root='./data2',
                                            train=True,
                                            transform = transform,
                                            download=False)

test_dataset = torchvision.datasets.CIFAR10(root='./data2',
                                           train=False,
                                           transform=transforms.ToTensor())

In [41]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [33]:
def conv3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsampling=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3(in_channels, out_channels, stride)
        self.batchnorm1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
#         Stride = 1 for every second block as we don't need downsampling.
        self.conv2 = conv3(out_channels, out_channels)
        self.batchnorm2 = nn.BatchNorm2d(out_channels)
        self.downsampling = downsampling
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batchnorm2(out)
        if self.downsampling:
            residual = self.downsampling(x)
        out += residual
        out = self.relu(out)
        return out

In [34]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3(3, 16)
        self.batchnorm = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):  
        downsampling = None
        if self.in_channels != out_channels or stride != 1:
            downsampling = nn.Sequential(conv3(self.in_channels, out_channels, stride=stride), 
                                         nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride=stride, downsampling=downsampling))
        self.in_channels = out_channels
#         residual_blocks.append(block(self.in_channels, out_channels))
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.batchnorm(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [35]:
model = ResNet(ResidualBlock, [2, 2, 2])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
decay = 0
model.train()
for epoch in range(epochs):
#     Decay the learning rate by a factor of 0.5 every 20 epochs
    if epoch+1%20 == 0:
        decay += 1 
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print(f"The new learning rate is {optimizer.param_groups[0]['lr']}")
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}, Loss: {loss.item():.4f}]")

Epoch [1/10], Step [100/500, Loss: 1.6692]
Epoch [1/10], Step [200/500, Loss: 1.4826]
Epoch [1/10], Step [300/500, Loss: 1.4013]
Epoch [1/10], Step [400/500, Loss: 1.0333]
Epoch [1/10], Step [500/500, Loss: 1.1044]
Epoch [2/10], Step [100/500, Loss: 1.0362]
Epoch [2/10], Step [200/500, Loss: 1.0260]
Epoch [2/10], Step [300/500, Loss: 0.8174]
Epoch [2/10], Step [400/500, Loss: 1.2424]
Epoch [2/10], Step [500/500, Loss: 0.9576]
Epoch [3/10], Step [100/500, Loss: 0.9696]
Epoch [3/10], Step [200/500, Loss: 0.7976]
Epoch [3/10], Step [300/500, Loss: 0.8118]
Epoch [3/10], Step [400/500, Loss: 0.7949]
Epoch [3/10], Step [500/500, Loss: 0.7994]
Epoch [4/10], Step [100/500, Loss: 0.7862]
Epoch [4/10], Step [200/500, Loss: 0.7192]
Epoch [4/10], Step [300/500, Loss: 0.6698]
Epoch [4/10], Step [400/500, Loss: 0.8473]
Epoch [4/10], Step [500/500, Loss: 0.9068]
Epoch [5/10], Step [100/500, Loss: 0.7788]
Epoch [5/10], Step [200/500, Loss: 0.8087]
Epoch [5/10], Step [300/500, Loss: 0.6519]
Epoch [5/10

In [37]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Accuracy of the model on the test images: {100*correct / total} %")

Accuracy of the model on the test images: 80.33 %
